In [1]:
import tkinter as tk
from PIL import Image, ImageTk
from tkinter import simpledialog

import random
from itertools import product
import time

### Card

In [2]:
# ranks:
# 1 -> Ace
# 2-10 -> 2-10
# 11 -> J
# 12 -> Q
# 13 -> K

class Card:
    def __init__(self, rank, suit):
        self.rank = rank
        self.suit = suit
        if self.rank == 1:
            self.card_scores = [1, 11]
        elif self.rank >= 11 and self.rank <= 13:
            self.card_scores = [10, 10]
        else:
            self.card_scores = [self.rank, self.rank]

        if self.rank == 1:
            self.short_rank = 'A'
        elif self.rank == 11:
            self.short_rank = 'J'
        elif self.rank == 12:
            self.short_rank = 'Q'
        elif self.rank == 13:
            self.short_rank = 'K'
        else:
            self.short_rank = str(self.rank)

        if self.suit == 'Spades':
            self.short_suit = 'S'
        elif self.suit == 'Hearts':
            self.short_suit = 'H'
        elif self.suit == 'Clubs':
            self.short_suit = 'C'
        else:
            self.short_suit = 'D'

        self.image_location = 'static/images/{}{}.png'.format(
            self.short_rank, self.short_suit)

    def __repr__(self):
        if self.rank == 1:
            true_rank = 'Ace'
        elif self.rank == 11:
            true_rank = 'Jack'
        elif self.rank == 12:
            true_rank = 'Queen'
        elif self.rank == 13:
            true_rank = 'King'
        else:
            true_rank = str(self.rank)
        return '{} of {}'.format(true_rank, self.suit)

suits = ('Spades', 'Hearts', 'Clubs', 'Diamonds')

### Deck

In [3]:
class Deck:
    def __init__(self, number_of_decks):
        self.number_of_decks = number_of_decks
        self.cards = []
        self.create(self.number_of_decks)
        self.count = 0

    def __repr__(self):
        return 'Game deck has {} cards remaining'.format(len(self.cards))

    def calculate_count(self):
        for card in self.cards:
            if card.rank in [1, 10, 11, 12, 13]:
                self.count += 1
            elif card.rank in [2, 3, 4, 5, 6]:
                self.count -= 1

    def create(self, number_of_decks):
        decks = [Card(rank, suit) for suit in suits for rank in range(1, 14)
                 for deck in range(number_of_decks)]
        decks = random.sample(decks, len(decks))
        self.cards.extend(decks)

    def create_test_deck(self,cards):
        self.cards = []
        for card in cards:
            self.cards.append(card)
    
    def draw(self):
        drawn_card = self.cards[0]
        self.cards.remove(self.cards[0])
        return drawn_card

    def reset(self):
        self.cards = []
        self.create(self.number_of_decks)

### Hand

In [22]:
class Hand:
    def __init__(self,rules,bet):
        self.hand_cards = []
        self.score = [0,0]
        self.options = []
        self.rules = rules
        self.hand_bet = bet
        self.blocked = False
        
    def calculate_score(self):
        # loop over all cards in hand
        self.score = [0,0]
        all_scores = []
        for card in self.hand_cards:
            all_scores.append(card.card_scores)
            
        self.score = sorted(list({sum(combination) for 
                                  combination in 
                                  product(*all_scores)}))[0:2]
        if len(self.score) < 2:
            self.score.append(self.score[0])
            
        #if self.score[0] == self.score[1]:
            #print('Current Hand score: {0}'.format(self.score[0]))
        #elif self.score[1] > 21:
            #print('Current Hand score: {0}'.format(self.score[0]))
        #else:
            #print('Current Hand score: {0} or {1}'.format(self.score[0], self.score[1]))

    def calculate_options(self):
        self.options = []
        if not self.blocked:
            if len(self.hand_cards) == 2: #initial possibilities after first draw
                if self.score[1] == 21:
                    return
                if self.rules.doublesoft:
                    if any(x in self.score for x in self.rules.doubledown):
                        self.options.append('doubledown')
                else:
                    if self.score[1] in self.rules.doubledown:
                        self.options.append('doubledown')
                if self.hand_cards[0].card_scores[0] == self.hand_cards[1].card_scores[0]:
                    self.options.append('split')
                if self.score[0] <= 21:
                    self.options.append('hit')
                self.options.append('stand')
            else: #after other actions have been taken
                if self.score[0] > 21:
                    self.options = ['BUST']
                if self.score[0] <= 21:
                    self.options.append('hit')
                    self.options.append('stand')
        #print(self.options)

    def calculate_hand_bet(self):
        print(self.hand_bet)
        
    def show_hand_cards(self):
        print(self.hand_cards)

    def hit(self,deck,num_cards):
        for i in range(0,num_cards):
            self.hand_cards.append(deck.draw())
        self.calculate_score()
        #self.show_hand_cards()
        self.calculate_options()
        #self.calculate_hand_bet()
        
    def stand(self):
        self.calculate_score()
        #self.show_hand_cards()
        self.options = ['none']
        #self.calculate_hand_bet()

    def doubledown(self,deck):
        self.hand_bet = 2*self.hand_bet
        self.hand_cards.append(deck.draw())
        self.calculate_score()
        if self.score[0] > 21:
            self.options = ['BUST']
        else:
            self.options = ['none']
        #self.show_hand_cards()
        #print(self.options)
        #self.calculate_hand_bet()
    
    def show_options(self):
        return self.options


### Player

In [33]:
class Player:
    def __init__(self,rules):
        self.funding = 0
        self.hands = []
        self.score = [0,0]
        self.rules = rules
        self.options = []

    def refund(self,funds):
        self.funding = funds
    
    def initiate(self, deck, bet):
        self.options = []
        self.bet = bet
        self.reset_hands()
        if self.add_hand():
            self.hands[0].hit(deck,2)
    
            if any(x in self.hands[0].score for x in [21]): #in case of blackjack no more options can be done
                self.options = ['blackjack']
                #print('blackjack on initial')

    def check_funds(self):
        #print('checking funds')
        if self.bet <= self.funding:
            return True
        else:
            return False
    
    def add_hand(self):
        if self.check_funds():
            self.update_funds()
            hand = Hand(self.rules, self.bet)
            self.hands.append(hand)
            return True
        else:
            print('insufficient funds')
            return False

    def update_funds(self):
        self.funding = self.funding - self.bet
        #print("funding after bet {}".format(self.funding))

    def show_options(self, hand_index):
        print(self.hands[hand_index].options)

    def hit(self, deck, hand_index):
        time.sleep(0.3)
        self.hands[hand_index].hit(deck,1)

    def stand(self, hand_index):
        self.hands[hand_index].stand()

    def doubledown(self, deck, hand_index):
        if self.check_funds:
            self.hands[hand_index].doubledown(deck)
            self.update_funds()
        else:
            print('insufficient funds for doubledown')

    def split(self, deck, hand_index):
        if self.check_funds:
            if len(self.hands) < self.rules.splitnumber:
                keep_card = self.hands[hand_index].hand_cards[0]
                transfer_card = self.hands[hand_index].hand_cards[1]
    
                hand = Hand(self.rules, self.bet)
                self.hands[hand_index].hand_cards = []
                self.hands.append(hand)
    
                index_new_hand = len(self.hands) - 1
    
                self.hands[hand_index].hand_cards.append(keep_card)
                self.hands[index_new_hand].hand_cards.append(transfer_card)

                if keep_card.rank == 1:
                    print('block splitted Aces')
                    self.hands[hand_index].blocked = True
                    self.hands[index_new_hand].blocked = True
                self.hands[hand_index].hit(deck, 1)
                self.hands[index_new_hand].hit(deck, 1)
                self.update_funds()
            else:
                print('maximum number of parallel hands reached')
        else:
            print('insufficient funds for split')
    
    def reset_hands(self):
        self.hands = []

### Dealer

In [29]:

class Dealer:
    def __init__(self,rules):
        self.hand_cards = []
        self.score = [0,0]
        self.options = []
        self.rules = rules
        
    def calculate_score(self, initial = False):
        # loop over all cards in hand
       
        if initial:
            self.score = [0,0]
            self.score[0] += self.hand_cards[0].card_scores[0]
            self.score[1] += self.hand_cards[0].card_scores[1]
        else:
            self.score = [0,0]
            all_scores = []
            for card in self.hand_cards:
                all_scores.append(card.card_scores)
                
            self.score = sorted(list({sum(combination) for 
                                      combination in 
                                      product(*all_scores)}))[0:2]
            if len(self.score) < 2:
                self.score.append(self.score[0])
        
        #if self.score[0] == self.score[1]:
            #print('Current Hand score: {0}'.format(self.score[0]))
        #elif self.score[1] > 21:
            #print('Current Hand score: {0}'.format(self.score[0]))
        #else:
            #print('Current Hand score: {0} or {1}'.format(self.score[0], self.score[1]))     
    
    def initiate(self, deck):
        self.reset_hand()
        self.hand_cards.append(deck.draw())
        self.hand_cards.append(deck.draw())

        self.calculate_score(initial = True)
        
        if self.hand_cards[0].rank in [1, 10, 11, 12, 13] and self.rules.peek:
            if self.hand_cards[0].card_scores[1] + self.hand_cards[1].card_scores[1] == 21:
                self.options = ['blackjack']
                #print(self.hand_cards)
            else:
                self.options = ['dealer ready']
                #print(self.hand_cards[0])
        else:
            self.options = ['dealer ready']
            #print(self.hand_cards[0])
        #print(self.options)
        
    def reveal(self):
        self.calculate_score()
        self.calculate_options()
    
    def hit(self, deck):
        self.hand_cards.append(deck.draw())

        self.calculate_score()
        self.calculate_options()
    
    def calculate_options(self):
        self.options = []
        if len(self.hand_cards) == 2 and self.score[1] == 21:
            self.options.append('blackjack')
        else:
            if self.score[0] == self.score[1]:
                if self.score[0] < 17:
                    self.options.append('hit')
                elif 17 <= self.score[0] <= 21:
                    self.options.append('none')
                else:
                    self.options.append('BUST')
            else:
                if 17 <= self.score[1] <= 21:
                    self.options.append('none')
                elif 17 <= self.score[0] <= 21:
                    self.options.append('none')
                elif self.score[0] > 21:
                    self.options.append('BUST')
                else:
                    self.options.append('hit')
        #print(self.options)
        
    def reset_hand(self):
        self.hand_cards = []

### Game

In [30]:
class Rules:
    def __init__(self):
        self.dealersoft17 = 'stand' #instert 'stand' or 'hit' for dealer on soft 17
        self.blackjack = 1.5 #insert multiplier in case of Blackjack on initial hand
        self.doubledown = [9,10,11] #insert ranks of cards that can be doubled
        self.doublesoft = False
        self.doubleaftersplit = True #insert if Player can double after split
        self.actionaftersplit = False #insert if Player can split Aces
        self.splitnumber = 3 #insert how many parallel sets Player can have
        self.peek = True
        self.num_of_decks = 1
        self.deck_penetration = 0.5
        self.test_deck = True

In [31]:
class Game:
    def __init__(self, rules, player, dealer, num_of_decks):
        self.rules = rules
        self.player = player
        self.dealer = dealer
        self.num_of_decks = num_of_decks
        self.deck = Deck(self.num_of_decks)
        self.result = []
        self.round_bet = 0
        self.reveal_dealer = False

    def check_penetration(self):
        if len(self.deck.cards) < self.num_of_decks*52*self.rules.deck_penetration:
            print('shuffle')
        else:
            print('{} cards left'.format(len(self.deck.cards)))

    def start_round(self, bet):
        self.eval = 0
        self.result = []
        self.reveal_dealer = False
        if 0 < bet <= self.player.funding:
            print('round started successfully')
            self.round_bet = bet
            self.player.initiate(self.deck, bet)
            self.dealer.initiate(self.deck)
    
            if 'blackjack' in self.dealer.options:
                if 'blackjack' in self.player.options:
                    self.result.append('push on initial')
                else:
                    self.result.append('player lost on dealer blackjack')
                self.evaluate()
            elif 'blackjack' in self.player.options:
                self.dealer.reveal()
                if self.dealer.score[1] == 21:
                    self.result.append('push on initial')
                    self.evaluate()
                else:
                    self.result.append('player won on initial blackjack')
                    self.dealer_turn()
            else:
                self.player_turn()
        else:
            print('insufficient funds')
            
    def player_turn(self):
        number_of_hands = len(self.player.hands)
        hand_count = 0
        for hand in self.player.hands:
            hand_index = self.player.hands.index(hand)
            if not any(x in ['BUST','none'] for x in hand.options):
                print('hand nr. {} waiting for input. current options: {}'.format(hand_index, hand.options))
            else:
                print('no more options for this hand')
                hand_count +=1
        if hand_count == number_of_hands:
            print('player has no more options')
            self.player.options = ['none']
            self.dealer_turn()
                
    def dealer_turn(self):
        self.dealer.reveal()
        self.reveal_dealer = True
        while 'hit' in self.dealer.options:
            self.dealer.hit(self.deck)
        if 'blackjack' in self.dealer.options:
            self.result.append('dealer reveals blackjack')
        self.evaluate()

    def calculate_counting_score(self, scores):
        if scores[0] == scores[1]:
            return scores[0]
        else:
            try:
                return max(score for score in scores if score <= 21)
            except:
                return 22
            

    def evaluate(self):
        self.eval += 1
        if self.eval == 1:
            print('evaluate nr.: {}'.format(self.eval))
            print('funds before eval: {}'.format(self.player.funding))
            print('dealer options: {}'.format(self.dealer.options))
            print('player options: {}'.format(self.player.options))
            for hand in self.player.hands:
                hand_index = self.player.hands.index(hand)
                print('player hand {} options: {}'.format(hand_index, hand.options))
            
            wins = 0
            if self.result:
                if 'player won on initial blackjack' in self.result:
                    win = (self.rules.blackjack * self.round_bet) + self.round_bet
                    wins += win
                    print('player blackjack:, win {}'.format(win))
                elif 'push on initial' in self.result:
                    win = self.round_bet
                    wins += win
                    print('push on initial:, win {}'.format(win))
                elif 'player lost on dealer blackjack' in self.result:
                    print('player lost on initial dealer blackjack')
                elif 'dealer reveals blackjack' in self.result:
                    print('player lost on late dealer blackjack')
            else:
                dealer_score = self.calculate_counting_score(self.dealer.score)
                for hand in self.player.hands:
                    hand_index = self.player.hands.index(hand)
                    hand_score = self.calculate_counting_score(hand.score)
                    print('evaluate hand {} hand score: {} dealer score: {}'.format(hand_index, hand_score, dealer_score))
    
                    if hand_score > 21:
                        print('hand {} BUST'.format(hand_index))
                    else:
                        if dealer_score > 21:
                            win = hand.hand_bet * 2
                            wins += win
                            print('hand {} wins, win {}'.format(hand_index, win))
                        else:
                            if hand_score < dealer_score:
                                print('hand {} lost'.format(hand_index))
                            if hand_score == dealer_score:
                                win = hand.hand_bet
                                wins += win
                                print('hand {} push, win {}'.format(hand_index, win))
                            if hand_score > dealer_score:
                                win = hand.hand_bet * 2
                                wins += win
                                print('hand {} wins, win {}'.format(hand_index, win))
            
            self.player.funding += wins
            print('funds after eval: {}'.format(self.player.funding))

In [34]:
class BettingApp:
    def __init__(self, root, test_deck):
        self.root = root
        self.buttons = []
        self.image_labels = []
        self.score_labels = []
        self.bet_labels = []
        self.rules = Rules()
        self.test_deck = test_deck
        self.setup_ui()
        

    def setup_ui(self):
        self.root.title("GUI with Placed Buttons")
        self.root.geometry("1200x900")
        self.load_background()

        self.funds = tk.StringVar(value="1000")
        self.bet = tk.StringVar(value="0")
        
        self.funds_label = tk.Label(self.root, textvariable=self.funds, font=('Arial', 16), bg='white')
        self.funds_label.place(relx=1.0, rely=0, x=-10, y=10, anchor='ne')

        self.bet_label = tk.Label(self.root, textvariable=self.bet, font=('Arial', 16), bg='lightblue')

        self.start_button = self.create_button("Start Game", self.start_game, 600, 450)
        self.rules_button = self.create_button("See Rules", self.see_rules, 50, 150, static = True)
        
        self.card_width = 80
        self.card_height = 116

    def load_background(self):
        bg_image_path = r"C:\Users\JanAllemann\repo\bj\static\visuals\table.jpg"
        try:
            bg_image = Image.open(bg_image_path)
            bg_image = bg_image.resize((1200, 900), Image.Resampling.LANCZOS)
            bg_photo = ImageTk.PhotoImage(bg_image)
            bg_label = tk.Label(self.root, image=bg_photo)
            bg_label.place(x=0, y=0, relwidth=1, relheight=1)
            self.bg_photo = bg_photo  # Keep a reference!
        except Exception as e:
            print(f"Failed to load background image: {e}")
    
    def create_button(self, text, command, x, y, static = False):
        button = tk.Button(self.root, text=text, command=command)
        button.place(x=x, y=y)
        if not static:
            self.buttons.append(button)
        return button

    def create_action_button(self, text, command, x, y):
        button = tk.Button(self.root, text=text, command=lambda: command())
        button.place(x=x, y=y)
        return button
    
    def wipe_button(self, button):
        button.place_forget()

    def wipe_all_buttons(self):
        for button in self.buttons:
            button.place_forget()

    def start_game(self):
        self.wipe_all_buttons()
        self.refund_button = self.create_button("Refund", self.refund, 50, 100)
        self.create_button("Deal", self.start_deal, 600, 450)
        self.create_button("Restart Game", self.setup_ui, 50 , 50, static = True)
        self.bet_label.place(x = 400, y = 800, anchor='ne')
        
        self.bet5_button = self.create_button("Bet +5", lambda: self.modify_bet(5), 50, 750)
        self.bet10_button = self.create_button("Bet +10", lambda: self.modify_bet(10), 120, 750)
        self.bet25_button = self.create_button("Bet +25", lambda: self.modify_bet(25), 190, 750)
        self.bet50_button = self.create_button("Bet +50", lambda: self.modify_bet(50), 260, 750)
        self.bet100_button = self.create_button("Bet +100", lambda: self.modify_bet(100), 330, 750)
        self.reset_bet_button = self.create_button("Reset Bet", self.reset_bet, 400, 750)

        self.player = Player(self.rules)
        self.dealer = Dealer(self.rules)
        self.game = Game(self.rules, self.player, self.dealer, num_of_decks = self.rules.num_of_decks)
        if self.rules.test_deck:
            self.game.deck.create_test_deck(self.test_deck)
        self.game.player.funding = float(self.funds.get())
        
        self.start_button.place_forget()
        print('game initiated')  # Assuming game initialization
    def show_shuffle_label(self):
        self.shuffle_label = tk.Label(self.root, text = "Reshuffle", bg = "white", fg = "black", font=('Arial', 48))
        self.shuffle_label.place(x = 600, y = 450, anchor = 'center')

        self.root.after(2000, self.shuffle_label.destroy)
        
    def next_round(self):
        if len(self.game.deck.cards) < self.rules.num_of_decks * 52 * self.rules.deck_penetration:
            print('reshuffle')
            self.game.deck = Deck(self.rules.num_of_decks)
            self.show_shuffle_label()
       
        self.wipe_all_buttons()
        self.refund_button = self.create_button("Refund", self.refund, 50, 100)
        self.create_button("Deal", self.start_deal, 600, 450)
        self.create_button("Restart Game", self.setup_ui, 50 , 50, static = True)
        self.bet_label.place(x = 400, y = 800, anchor='ne')
        
        self.bet5_button = self.create_button("Bet +5", lambda: self.modify_bet(5), 50, 750)
        self.bet10_button = self.create_button("Bet +10", lambda: self.modify_bet(10), 120, 750)
        self.bet25_button = self.create_button("Bet +25", lambda: self.modify_bet(25), 190, 750)
        self.bet50_button = self.create_button("Bet +50", lambda: self.modify_bet(50), 260, 750)
        self.bet100_button = self.create_button("Bet +100", lambda: self.modify_bet(100), 330, 750)
        self.reset_bet_button = self.create_button("Reset Bet", self.reset_bet, 400, 750)
        print('round initiated')
        
    def see_rules(self):
        text_widget = tk.Text(self.root, width=40, height=15)
        text_widget.pack()
    
        # Inserting each attribute and its value into the Text widget
        for attr, value in self.rules.__dict__.items():
            text_widget.insert(tk.END, f"{attr}: {value}\n")
        self.hide_rules_button = self.create_button("Hide Rules", lambda: self.hide_rules(text_widget), 50, 150)

    def hide_rules(self, widget):
        widget.pack_forget()
        self.hide_rules_button.place_forget()
        self.rules_button = self.create_button("See Rules", self.see_rules, 50, 150, static = True)
            
    def refund(self):
        self.funds.set("0")
        response = simpledialog.askinteger("Input", "Enter refund amount:", parent=self.root)
        if response is not None:
            current_funds = float(self.funds.get())
            self.funds.set(str(current_funds + response))
            print(f"Refund amount set to: {response}, Total funds: {self.funds.get()}")
            self.game.player.funding = float(self.funds.get())
        else:
            print("No refund amount entered")
    
    def start_deal(self):
        if int(self.bet.get()) > 0:
            self.game.start_round(int(self.bet.get()))
            self.funds.set(str(self.game.player.funding))
            self.wipe_all_buttons()
            self.bet_label.place_forget()

            if any(x in ['push on initial','player lost on dealer blackjack'] for x in self.game.result):
                self.button_end_round = self.create_button('End round', self.end_round, 600, 450)
                self.game.reveal_dealer = True
                self.read_and_display_cards()
            else:    
                self.read_and_display_cards(initial_deal = True)
                self.start_player_input()
        else:
            print("bet can't be 0")

    def end_round(self):
        self.funds.set(str(self.game.player.funding))
        self.wipe_images()
        self.wipe_scores()
        self.wipe_bets()
        self.next_round()
    
    def read_and_display_cards(self, initial_deal = False):
        dealer_score = []
        if not self.game.reveal_dealer:
            x_card = 0
            for card in self.game.dealer.hand_cards[0:1]:
                self.display_card(card = card, x = 600 + x_card, y = 150)
                dealer_score.append(card.card_scores)
                x_card += 50
        else:
            x_card = 0
            for card in self.game.dealer.hand_cards:
                self.display_card(card = card, x = 600 + x_card, y = 150)
                dealer_score.append(card.card_scores)
                x_card += 50
        self.display_score(dealer_score, 600, 300)
                
        x_hand = 0
        for hand in self.player.hands:
            hand_index = self.player.hands.index(hand)
            hand_score = []
            x_card = 0
            for card in hand.hand_cards:
                self.display_card(card = card, x = (150 + x_hand) + x_card, y = 450)
                x_card += 50
                hand_score.append(card.card_scores)
            self.display_score(hand_score, 150 + x_hand, 600)
            self.display_bet(hand.hand_bet, 150 + x_hand, 650)
            x_hand += 300
                
    def display_card(self, card, x, y):
        card_image = Image.open(card.image_location)
        card_image = card_image.resize((self.card_width, self.card_height), Image.Resampling.LANCZOS)
        # Convert the image for tkinter
        tk_photo = ImageTk.PhotoImage(card_image)
        
        # Create a label to display the image
        label = tk.Label(self.root, image=tk_photo)
        label.image = tk_photo  # Keep a reference!
        label.place(x=x, y=y)
        self.image_labels.append(label)

    def wipe_scores(self):
        for label in self.score_labels:
            label.place_forget()
        self.score_labels = []
        
    def display_score(self, all_scores, x, y):
        score = []
        score = sorted(list({sum(combination) for 
                                      combination in 
                                      product(*all_scores)}))[0:2]
        if len(score) < 2:
            score.append(score[0])
        
        if score[0] == score[1]:
            score = str(score[0])
        elif score[1] > 21:
            score = str(score[0])
        else:
            score = str(score[0]) + 'or' + str(score[1])
        label = tk.Label(self.root, text = score, bg = "white", fg = "black", font=('Arial', 16))
        label.place(x = x, y = y)
        self.score_labels.append(label)

    def wipe_bets(self):
        for label in self.bet_labels:
            label.place_forget()
        self.bet_labels = []
        
    def display_bet(self, bet, x, y):
        label = tk.Label(self.root, text = str(bet), bg = "white", fg = "black", font=('Arial', 16))
        label.place(x = x, y = y)
        self.bet_labels.append(label)        
        
    def wipe_images(self):
        for label in self.image_labels:
            label.place_forget()
        self.image_labels = []

    def read_and_display_options(self, hand, hand_index):
        options = []
        for option in hand.options:
            if option == "hit":
                self.create_button("Hit" + str(hand_index), lambda: self.hit(hand_index), hand_index*300 + 150, 700)
                options.append('hit')
            if option == "stand":
                self.create_button("Stand" + str(hand_index), lambda: self.stand(hand_index), hand_index*300 + 150, 750)
                options.append('stand')
            if option == "doubledown":
                self.create_button("DD" + str(hand_index), lambda: self.doubledown(hand_index), hand_index*300 + 150, 800)
                options.append('doubledown')
            if option == "split":
                self.create_button("Split" + str(hand_index), lambda: self.split(hand_index), hand_index*300 + 150, 850)
                options.append('split')
        return options
        #if num_of_options == 0:
         #   self.button_end_round = self.create_button('End round', self.end_round, 600, 450)

    def start_player_input(self):
        self.current_hand_index = 0  # Start with the first hand
        self.process_next_hand()
    def process_next_hand(self):
        if self.current_hand_index < len(self.game.player.hands):
            self.get_player_input(self.current_hand_index)
        else:
            print("All hands processed.")
            # Handle end of all hand processing here, such as showing end round button
            self.game.dealer_turn()
            self.read_and_display_cards()
            self.button_end_round = self.create_button('End round', self.end_round, 600, 450)
    def get_player_input(self, hand_index):
        hand = self.game.player.hands[hand_index]
        options = self.read_and_display_options(hand, hand_index)
        if len(options) == 0:  # If no options, move to the next hand
            print('options detected: {}'.format(options))
            self.current_hand_index += 1
            self.process_next_hand()
    def continue_game(self, hand_index):
        self.wipe_all_buttons()    
        self.read_and_display_cards()
        self.get_player_input(hand_index)
    def hit(self, hand_index):
        print(f'Player hits hand {hand_index}')
        self.game.player.hit(self.game.deck, hand_index)
        self.continue_game(hand_index)
    def stand(self, hand_index):
        print(f'Player stands hand {hand_index}')
        self.game.player.stand(hand_index)
        self.continue_game(hand_index)
    def doubledown(self, hand_index):
        print(f'Player doubles hand {hand_index}')
        self.game.player.doubledown(self.game.deck, hand_index)
        self.funds.set(str(self.game.player.funding))
        self.continue_game(hand_index)
    def split(self, hand_index):
        print(f'Player splits hand {hand_index}')
        self.game.player.split(self.game.deck, hand_index)
        self.funds.set(str(self.game.player.funding))
        self.continue_game(hand_index)

    # Function to modify bet
    def modify_bet(self, amount):
        current_bet = int(self.bet.get())
        current_funds = float(self.funds.get())
        if current_bet + amount <= current_funds:  # Check if funds are sufficient
            self.bet.set(str(current_bet + amount))
        else:
            print("Not enough funds to increase bet")
    
    def reset_bet(self):
        self.bet.set("0")

test_deck = [Card(1,'Hearts'),Card(1,'Hearts'),Card(10,'Spades'),Card(10,'Diamonds'),Card(8,'Spades'),Card(3,'Spades'),Card(11,'Spades'),Card(11,'Spades'),Card(11,'Spades'),Card(11,'Spades'),Card(11,'Spades'),Card(11,'Spades')]
app = BettingApp(tk.Tk(), test_deck)
app.root.mainloop()

game initiated
round started successfully
hand nr. 0 waiting for input. current options: ['split', 'hit', 'stand']
Player splits hand 0
block splitted Aces
options detected: []
options detected: []
All hands processed.
evaluate nr.: 1
funds before eval: 980.0
dealer options: ['none']
player options: []
player hand 0 options: []
player hand 1 options: []
evaluate hand 0 hand score: 19 dealer score: 20
hand 0 lost
evaluate hand 1 hand score: 14 dealer score: 20
hand 1 lost
funds after eval: 980.0
reshuffle
round initiated
round started successfully
hand nr. 0 waiting for input. current options: ['hit', 'stand']


In [95]:
app.game.player.hands